<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/LMS/nlp_node8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About this notebook  
이 노트북은 모두의 연구소 AIFFEL LMS NLP node8의 프로젝트입니다. seq2seq 한-영 번역기를 attention을 사용하여 만듭니다.   

This is written as a part of the deep learning training course by Aiffel, Korea. LMS NLP node 8. I have to make a korean - english seq2seq translator. Additionaly attention should be used.

In [1]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
import os
import io

# Checkout version of libraries

```python
import pandas as pd
import matplotlib

print(pd.__version__)
print(tf.__version__)
print(matplotlib.__version__)
```

# Download, load dataset
download korean-english-park.train.tar.gz from this github rep.
[jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)  
instead of tf.keras.utils.get_file(), google drive web page and terminal used.

In [2]:
en_path = '/content/drive/MyDrive/colabdata/modulabs/nlp8/korean-english-park.train.en'
ko_path = '/content/drive/MyDrive/colabdata/modulabs/nlp8/korean-english-park.train.ko'
f_ko=open(ko_path, 'r')
f_en=open(en_path, 'r')
ko_raw = f_ko.readlines()
en_raw = f_en.readlines()

In [3]:
print(len(ko_raw), len(en_raw))

94123 94123


In [4]:
data = pd.DataFrame()
test_rows=0 #temporal index for scripting
if test_rows:
    data['ko_raw']=ko_raw[:test_rows]
    data['en_raw']=en_raw[:test_rows]
else:
    data['ko_raw']=ko_raw
    data['en_raw']=en_raw
data

,ko_raw,en_raw
0,"개인용 컴퓨터 사용의 상당 부분은 ""이것보다 뛰어날 수 있느냐?""\n","Much of personal computing is about ""can you t..."
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,so a mention a few weeks ago about a rechargea...
2,그러나 이것은 또한 책상도 필요로 하지 않는다.\n,"Like all optical mice, But it also doesn't nee..."
3,"79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분...",uses gyroscopic sensors to control the cursor ...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔...,Intelligence officials have revealed a spate o...
...,...,...
94118,“우리는 3월 8일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 ...,””We are hoping to seize material evidence to ...
94119,월요일 술집 종업원 6명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했...,"” On Monday, police secured statements from si..."
94120,그러나 불충분한 증거 확보로 수사에 어려움이 있다.\n,But the lack of material evidence is making it...
94121,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다.\n,Kim and his son both deny the allegations.\n


# About Data load...

두 파일은 영어, 한국어 문장이 순서대로 들어가있으므로 순서를 그대로 읽어야 합니다. 
pandas.read_csv나 read_table을 썼을 때에는 줄의 개수가 달라져서 seperator를 고쳐보다가 안돼서 그냥 file.readlines()를 사용하였습니다.

Two data file have corresponding sentences. So its order should be preserved. but when I have tried pandas.read_csv and pandas.read_table first. In that case, length of both file did not matched. maybe I should change seperator   



# Data refine, cleaning

In [5]:
data.isna().any()

ko_raw    False
en_raw    False
dtype: bool

In [6]:
#data['ko_raw'].apply(lambda x: '\\' in x)

## Characters?
 before we process regular expressions, let's figure out set of characters shown in each dataset.

In [7]:
def characters_in_column(df, col_name):
    """characters_in_column(df, col_name)
    df is pandas dataframe. this function converts df["col_name"] into set of characters. return ordered list.
    """
    _=pd.Series()
    _=df[col_name].apply(lambda x: [i for i in x])
    return np.unique(np.concatenate(_.values))


In [8]:
ko_chars=characters_in_column(data, 'ko_raw')
print(ko_chars)
print(' '.join(ko_chars))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


['\n' ' ' '!' ... '８' '？' '\U000d51d1']

   ! " # $ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; = > ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z { } ~   ± ´ · × é ˙ ˝ – ― ‘ ’ “ ” • ‥ … ℃ ℓ ▲ ▶ 　 〈 〉 〔 〕 い か き く さ ざ し す そ て と ば ぶ ぷ ま や よ ら り ん ㄴ ㅇ ㆍ ㈜ ㎝ ㎞ ㎠ ㎡ ㎢ ㎾ 一 万 三 上 不 丙 中 主 之 九 亞 交 京 仁 代 佛 促 信 修 假 備 傷 像 僞 價 億 兆 光 克 兒 內 全 公 兵 再 切 前 劉 力 加 動 勝 北 區 卍 占 印 卿 反 受 古 司 吉 同 名 吳 命 和 品 商 善 喩 器 四 因 國 園 圓 團 在 地 坤 型 城 基 堀 報 場 塔 墓 壽 外 多 大 天 夫 奔 奴 姓 婦 子 字 孩 學 宇 安 官 室 家 富 察 寧 寶 寺 對 導 小 局 居 屋 山 岩 島 峽 川 州 市 席 常 平 幹 店 度 座 庫 康 廣 建 式 弗 强 影 後 心 性 愁 愚 愛 慶 懷 戰 所 手 掃 掌 換 播 故 效 敎 敢 敵 文 斌 新 方 族 旗 日 早 明 星 時 晉 晴 書 曾 會 月 本 朱 李 東 林 査 株 案 棺 植 楊 機 權 次 止 正 死 殘 毁 母 毛 毬 民 氣 水 氷 江 沈 河 油 法 泥 泰 流 浅 海 淘 淺 港 湖 湛 溫 滑 澤 濠 濤 濱 瀋 灣 火 無 然 焼 煙 照 燭 父 物 獨 獸 率 王 珀 現 球 理 琥 環 生 産 田 男 異 畵 疑 疫 病 登 發 的 省 眞 石 社 神 票 祭 福 私 種 空 突 章 管 節 築 精 紅 紙 結 總 纏 置 羅 美 群 習 翔 老 者 耳 聖 聲 職 肢 胡 能 膚 臺 舊 航 船 花 芽 英 草 華 董 葬 蓄 蓋 蔣 藥 藻 號 街 衛 要 視 親 覺 言 詩 誌 語 談 論 諡 諸 識 貧 買 費 賣 質 赤 走 超 足 路 身 軌 

온갖 글자가 다 있습니다. 줄넘김, 한자도 있네요.

## 텍스트 전처리 수정(정제)
노드의 코드를 보면 정규표현식을 다음과 같이 사용합니다.
```python
sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
sentence = re.sub(r'[" "]+', " ", sentence)
sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
```
위의 예에서 보면 많은 경우 \n 등이 삽입돼 있는데 줄넘김 표시이므로 삭제합니다.  
외래어도 자연스럽게 우리말 문장에 영어로 들어가 있는 경우도 많 때문에 굳이 영어 알파벳를 빼지는 않겠습니다.

```python
sentence = re.sub(r"\\[a-zA-Z]", " ", sentence)
sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
sentence = re.sub(r'[" "]+', " ", sentence)
sentence = re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z?.!,]+", " ", sentence)
```

In [9]:
#print(re.sub(r"\\[a-zA-Z]", " ", '\dsdfasf dasfd ahf'))
#print('\ndasd')

In [10]:
def preprocess_en(sentence, s_token=True, e_token=True):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"\\[a-zA-Z]", " ", sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

In [11]:
def preprocess_ko(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"\\[a-zA-Z]", " ", sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [12]:
data['ko_sentence']=data['ko_raw'].apply(preprocess_ko)
data['en_sentence']=data['en_raw'].apply(preprocess_en)
data.drop(columns=['en_raw','ko_raw'], inplace=True)
data

,ko_sentence,en_sentence
0,개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?,<start> much of personal computing is about ca...
1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,<start> so a mention a few weeks ago about a r...
2,그러나 이것은 또한 책상도 필요로 하지 않는다 .,"<start> like all optical mice , but it also do..."
3,". 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이...",<start> uses gyroscopic sensors to control the...
4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을...,<start> intelligence officials have revealed a...
...,...,...
94118,우리는 월 일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 총력을...,<start> we are hoping to seize material eviden...
94119,월요일 술집 종업원 명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했다 .,"<start> on monday , police secured statements ..."
94120,그러나 불충분한 증거 확보로 수사에 어려움이 있다 .,<start> but the lack of material evidence is m...
94121,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다 .,<start> kim and his son both deny the allegati...


## 중복 데이터 삭제

In [13]:
data[data.duplicated(keep=False)].sort_values(by='ko_sentence')

,ko_sentence,en_sentence
69441,", kg 에메랄드 소유권 분쟁",<start> pound emerald at center of dispute <end>
69429,", kg 에메랄드 소유권 분쟁",<start> pound emerald at center of dispute <end>
86489,", korean women are working as prostitutes in j...",<start> some of the brokers bragged about bein...
73805,", korean women are working as prostitutes in j...",<start> some of the brokers bragged about bein...
62240,", 고대 이집트 지배층 유물 전시회","<start> king tut and , years in ancient egypt ..."
...,...,...
93792,"힐러리 클린턴 , 북한 방문할까",<start> would hillary clinton visit north kore...
79517,"힐러리 클린턴 , 북한 방문할까",<start> would hillary clinton visit north kore...
78621,"힐러리 클린턴 , 북한 방문할까",<start> would hillary clinton visit north kore...
64666,힐러리 클린턴은 지난 년 월 이후 상원의원에 재임 중이다 .,<start> hillary clinton has been in the senate...


In [14]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)
data

,index,ko_sentence,en_sentence
0,0,개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?,<start> much of personal computing is about ca...
1,1,모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하...,<start> so a mention a few weeks ago about a r...
2,2,그러나 이것은 또한 책상도 필요로 하지 않는다 .,"<start> like all optical mice , but it also do..."
3,3,". 달러하는 이 최첨단 무선 광마우스는 허공에서 팔목 , 팔 , 그외에 어떤 부분이...",<start> uses gyroscopic sensors to control the...
4,4,정보 관리들은 동남 아시아에서의 선박들에 대한 많은 테러 계획들이 실패로 돌아갔음을...,<start> intelligence officials have revealed a...
...,...,...,...
78840,94118,우리는 월 일 김승연 회장과 그의 아들이 보복폭행에 가담한 혐의를 찾기 위해 총력을...,<start> we are hoping to seize material eviden...
78841,94119,월요일 술집 종업원 명은 김회장과 아들에게 폭행을 당했음을 진술했다고 경찰은 말했다 .,"<start> on monday , police secured statements ..."
78842,94120,그러나 불충분한 증거 확보로 수사에 어려움이 있다 .,<start> but the lack of material evidence is m...
78843,94121,김회장과 그의 아들은 보복폭행 혐의를 강력히 부인하고 있다 .,<start> kim and his son both deny the allegati...


In [15]:
data.drop(columns=['index'], inplace=True)

데이터 중복 문제가 심각하네요. set를 사용하라고 했는데 그냥 pandas의 drop_duplicate를 썼습니다.

## 토큰화 수정
한국어의 형태소 분석 적용 토큰화: MeCab을 사용합니다. 이것은 [exploration node 8(github)](https://github.com/chhyyi/aiffel/blob/main/LMS/EXP8_imdb_emotion_analysis.ipynb)에서 했던 것을 참고하여 작성합니다.


In [16]:
!apt-get update && apt-get install -y g++ default-jdk
!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [17]:
import konlpy
import gensim
from konlpy.tag import Mecab

num_words=10000
tokenizer = Mecab()
stopwords_kr = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [18]:
from collections import Counter
#Counter([1,2,3,4,2,3,4])
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
en_stopwords = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def tokenize_and_int_encode(df, num_words=num_words, maxlen=40):
    """
    tokenize_and_int_encode(df, num_words=num_words, maxlen=40)
    receive pandas DataFrame and tokenize it's ko_sentence.
    return processed korean sentences and corresponding english sentences
    return dictionaries; word_to_index, index_to_word
    """
    idx=[] #integer encoded sentences array. will be returned first.
    
    raw_corpus=[] #after Counter, set(raw_corpus) will be corpus.
    en_sentences=[]
    ko_sentences=[]
    #word to_index, index_to_word 초기화.

    word_to_index={}
    index_to_word={}
    #dict_last_index=3 #while 0, 1, 2 will be <PAD>, <BOS>, <UNK>(unkown)

    #tokenize by mecab
    for ko_sent, en_sent in zip(df['ko_sentence'].to_list(), df['en_sentence'].to_list()):
        tokens=list(tokenizer.morphs(ko_sent))
        if len(tokens)>maxlen:
            continue
        else:
            en_sentences.append(en_sent)
            ko_sentences.append(tokens)
            raw_corpus+=tokens
    raw_corpus = [ww for ww in raw_corpus if ww not in stopwords_kr]
    print('example of raw_corpus',raw_corpus[:10])

    most_words=dict(Counter(raw_corpus).most_common(num_words)).keys()
    print('length of most common {} words are '.format(num_words),len(most_words))
    word_to_index = dict(zip(most_words, list(range(3, 3+len(most_words)))))
    index_to_word = dict(zip(list(range(3, 3+len(most_words))), most_words))

    #encode as integer
    for tokens in ko_sentences:
        _=[]
        for tk in tokens:
            if tk in most_words:
                _.append(word_to_index[tk])
            else:
                _.append(2)
        idx.append(_)

    word_to_index={**word_to_index, "<PAD>":0, "<BOS>":1, "<UNK>":2}
    index_to_word={**index_to_word, 0:"<PAD>", 1:"<BOS>", 2:"<UNK>"}                

    return idx, en_sentences, word_to_index, index_to_word

#index_to_word도 load_data에서 만들었기 때문에 그냥 같이 반환합니다.
kr_idx, en_sentences, word2index_ko, index2word_ko=tokenize_and_int_encode(data)

example of raw_corpus ['개인', '용', '컴퓨터', '사용', '상당', '부분', '이것', '보다', '뛰어날', '수']
length of most common 10000 words are  10000


In [20]:
print('lengths - kr_idx(enc_input):{}, en_sentences:{}, word2index_ko:{}, index2word_ko:{}'.format(len(kr_idx), len(en_sentences), len(word2index_ko), len(index2word_ko)))

lengths - kr_idx(enc_input):66360, en_sentences:66360, word2index_ko:10003, index2word_ko:10003


In [21]:
data['en_sentence'].sample().values

array(['<start> a three way meeting with u . s . secretary of state condoleezza rice , israeli prime minister ehud olmert and palestinian president mahmoud abbas will take place in jerusalem on february , olmert said during a speech tuesday . <end>'],
      dtype=object)

In [22]:
def en_tokenize_and_encode(en_sents, num_words=num_words):
    """
    tokenize_and_int_encode
    take english sentences and tokenize and encode as integers.
    return processed english sentences
    return dictionaries; word_to_index, index_to_word
    """

    idx=[] #integer encoded sentences array. will be returned first.

    raw_corpus=[] #after Counter, set(raw_corpus) will be corpus.
    sentences=[]
    #word to_index, index_to_word 초기화.

    word_to_index={}
    index_to_word={}
    #dict_last_index=3 #while 0, 1, 2 will be <PAD>, <BOS>, <UNK>(unkown)

    #tokenize by mecab
    for sent in en_sents:
        sentences.append(list(sent.split()))
        raw_corpus+=sentences[-1]
    raw_corpus=[i for i in raw_corpus if i not in en_stopwords]
    most_words=list(dict(Counter(raw_corpus).most_common(num_words)).keys())
    print('length of most common {} words are '.format(num_words),len(most_words))
    print('most common words: ', most_words[:10])
    word_to_index = dict(zip(most_words, list(range(3, 3+len(most_words)))))
    index_to_word = dict(zip(list(range(3, 3+len(most_words))), most_words))
    print(len(word_to_index), len(index_to_word))

    for tokens in sentences:
        _=[]
        for tk in tokens:
            if tk in most_words:
                _.append(word_to_index[tk])
            else:
                _.append(2)
        idx.append(_)

    word_to_index={**word_to_index, "<PAD>":0, "<BOS>":1, "<UNK>":2}
    index_to_word={**index_to_word, 0:"<PAD>", 1:"<BOS>", 2:"<UNK>"}                

    return idx, word_to_index, index_to_word


In [23]:
en_idx, word2index_en, index2word_en=en_tokenize_and_encode(en_sentences)

length of most common 10000 words are  10000
most common words:  ['.', ',', '<start>', '<end>', 'said', 'year', 'people', 'two', 'u', 'one']
10000 10000


In [24]:
#np.max([len(x) for x in en_idx])

In [25]:
en_idx = tf.keras.preprocessing.sequence.pad_sequences(en_idx, padding='post')
kr_idx = tf.keras.preprocessing.sequence.pad_sequences(kr_idx, padding='post')

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(kr_idx, en_idx, test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(53088, 40) (13272, 40) (53088, 85) (13272, 85)


In [27]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.d1 = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units, return_sequences = True)
		
    def call(self, x):
        x = self.d1(x)
        x=self.gru(x)
        return x


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units, return_sequences=True, return_state=True)
        self.attention = BahdanauAttention(self.dec_units)   # Attention 필수 사용!
        self.output_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, x, h_dec, enc_out):
        context_vec, attn =self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis = -1)

        out, state = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.output_layer(out)

        return out, h_dec, attn

# 코드를 실행하세요.

BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(index2word_ko) + 1
TGT_VOCAB_SIZE = len(index2word_en) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

print("슝~")

@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_w2i):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_w2i['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

from tqdm import tqdm    # tqdm
import random

EPOCHS = 10

@tf.function
def eval_step(src, tgt, encoder, decoder, optimizer, dec_w2i):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_w2i['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))
    return batch_loss


for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, X_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)    # tqdm

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(X_train[idx:idx+BATCH_SIZE],
                                Y_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                word2index_en)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))    # tqdm
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))    # tqdm
    
    test_total_loss = 0
    idx_list = list(range(0, X_test.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = eval_step(X_test[idx:idx+BATCH_SIZE], Y_test[idx:idx+BATCH_SIZE],
        encoder, decoder,
        optimizer, word2index_en)
        total_loss +=batch_loss

        t.set_description_str('Test Epoch %2d' % (epoch + 1))    # tqdm
        t.set_postfix_str('Test Loss %.4f' % (total_loss.numpy() / (batch + 1)))    # tqdm

Encoder Output: (64, 30, 1024)
Decoder Output: (64, 10004)
Decoder Hidden State: (64, 1024)
Attention: (64, 30, 1)
슝~


Test Epoch 10: 100%|██████████| 208/208 [01:52<00:00,  1.85it/s, Test Loss 4.3510]


In [66]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((Y_train.shape[-1], X_train.shape[-1]))
    
    ####inserted line by chyi####
    ko_sent = preprocess_ko(sentence)
    _=pd.DataFrame()
    
    tokens=tokenizer.morphs(ko_sent)

    _=[]
    for tk in tokens:
        if tk in word2index_ko.keys():
            _.append(word2index_ko[tk])
        else:
            _.append(2)
    inputs=[_,]
    print(inputs)
    
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=40,
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([word2index_en['<start>']],0)

    for t in range(Y_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()
        #if predicted_id>2:
        result += index2word_en[predicted_id] + ' '

        if index2word_en[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    #attention = attention[:len(result.split()), :len(sentence.split())]
    #plot_attention(attention, sentence.split(), result.split(' '))


translate("나는 그런 한심한 상황을 이해할 수 없었다. 왜 그렇게 된 거지?", encoder, decoder)


[[83, 2, 1303, 2, 2, 251, 5, 1734, 26, 29, 56, 21, 4, 3, 1686, 1973, 31, 2009, 19, 349]]
Input: 나는 그런 한심한 상황을 이해할 수 없었다. 왜 그렇게 된 거지?
Predicted translation: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 


In [67]:
path_save = '/content/drive/MyDrive/colabdata/modulabs/nlp8/'
encoder.save_weights(path_save+'my_checkpoint_enc')
decoder.save_weights(path_save+'my_checkpoint_dec')

In [68]:
translate('개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?', encoder, decoder)

[[696, 596, 522, 160, 2, 1312, 840, 2, 707, 150, 2, 29, 9, 3709, 349]]
Input: 개인용 컴퓨터 사용의 상당 부분은 이것보다 뛰어날 수 있느냐 ?
Predicted translation: <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 


In [54]:
word2index_en['<start>']

5